# Search Results Files

In [ ]:
%matplotlib inline
import os

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import plotly.express as px
import zipfile

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
import itables.options as opt

import matplotlib as mpl

opt.maxBytes = 0
opt.classes = ["display", "nowrap","compact","hover"]
opt.showIndex = False
opt.style = "max-width:6000px"
pd.set_option('display.max_colwidth', 400)

In [ ]:
! wget -q --no-parent https://genesis.ugent.be/uvpublicdata/copenhagen_20220913/ionbot_teeth_copenhagen.zip ./
! mkdir ~/ionbot_result_files/
! mkdir ~/ionbot_processed_result_files/
! unzip -q ionbot_teeth_copenhagen.zip -d ~/ionbot_result_files

The `ionbot.twbx` result file contains the matched peptides and proteins as a compressed file. 

This file can be renamed to better relfect the processed sample.

In the following field you can specify the path to the result file: 

In [ ]:
twbx_file = "./ionbot_result_files/20150929_QE5_UPLC10_RJC_SA_Plaque6_01/ionbot.twbx"

This result file can be extracted as a zip file, but here we will decompress the file using Python.

You can specify the folder to where to extract the ionbot result files to:

In [ ]:
main_result_folder = "./ionbot_processed_result_files/"
main_download_folder = "./ionbot_result_files/"

In [ ]:
ionbot_results_dir = "./ionbot_result_files/"
processed_files = []

for f in os.listdir(ionbot_results_dir):
    if os.path.isfile(os.path.join(ionbot_results_dir,f)):
        continue
        
    archive = zipfile.ZipFile(os.path.join(os.path.join(main_download_folder,f),"ionbot.twbx"))
    
    for file in archive.namelist():
        if file.startswith('Data/'):
            archive.extract(file, os.path.join(main_result_folder,f))
    processed_files.append(os.path.join(main_result_folder,f))

The result files are written to the subfolders `Data/ionbot_result`:

In [ ]:
result_folder = [os.path.join(pf,"Data/ionbot_result") for pf in processed_files]

The content of the result files is described [here](https://ionbot.cloud/help).

## The PSM results

First, we load the result file that contains the first ranked matches for each MS2 spectrum:

In [ ]:
single_result_folder = "ionbot_processed_result_files/20150929_QE5_UPLC10_RJC_SA_Plaque6_01/Data/ionbot_result/"
ionbot = pd.read_csv(os.path.join(single_result_folder,"ionbot.first.csv"))

These are the column names:

In [ ]:
for col in ionbot.columns:
    print(col)

Let's print some columns and explain the content:

In [ ]:
cols_to_use = ["ionbot_match_id","database_peptide","matched_peptide",
               "modifications","modifications_delta","unexpected_modification"]
ionbot[cols_to_use]

The column `database` is `T` if the PSM matched the target database, it is `D` otherwise.

In [ ]:
cols_to_use = ["ionbot_match_id","database","q-value"]
ionbot[cols_to_use]

We can see that the result file contains all matches with FDR <= 1%:

In [ ]:
print(ionbot["database"].value_counts())

The column `psm_score` contains the PSM score for the matched spectra:

In [ ]:
px.histogram(ionbot,
             x="psm_score", 
             color="database", 
             nbins=50
            )

Next, we load the result file that contains the lower ranked (co-eluting) matches for each MS2 spectrum and add these to the search results:

In [ ]:
ionbot["rank"] = ["first"]*len(ionbot)
tmp = pd.read_csv(os.path.join(single_result_folder,"ionbot.lower.csv"))
tmp["rank"] = ["lower"]*len(tmp)
ionbot = pd.concat([ionbot,tmp])

For the remainder, we remove the matches against the decoy database:

In [ ]:
ionbot = ionbot[(ionbot["database"]=="T")]

While adding the lower ranked matches we created a column `rank` that has value 'first' if the match was ranked first based on the psm_score, and 'lower' otherwise:

In [ ]:
print(ionbot["rank"].value_counts())

To reconstruct the LC-MS separation for matched MS2 spectra we can use the `observed_retention_time` and `precursor_mass` columns: 

In [ ]:
fig = px.scatter(ionbot, 
                 x="observed_retention_time", 
                 y="precursor_mass", 
                 color="rank",
                 hover_data=["ionbot_match_id","matched_peptide"]
                )
fig.update_traces(marker=dict(size=2))
fig.show()

The `ionbot.features.csv` result files contains the matching information used in the PSM scoring function.

We load `ionbot.features.csv` and merge it with the search results: 

In [ ]:
os.path.join(single_result_folder,"/ionbot.features.csv")

In [ ]:
features = pd.read_csv(f"{single_result_folder}/ionbot.features.csv")
ionbot = ionbot.merge(features,on="ionbot_match_id",how="left")

for col in features.columns:
    print(col)

We can plot these feature values as boxplots:

In [ ]:
px.box(ionbot, 
       y=["by-count","all-count"],
       color="rank",
       hover_data=["ionbot_match_id"]
      )

In [ ]:
px.box(ionbot, 
       y=["by-explained","all-explained"],
       color="rank",
       hover_data=["ionbot_match_id"]       
      )

In [ ]:
px.box(ionbot, 
       y=["by-intensity-pattern-correlation"],
       color="rank",
       hover_data=["ionbot_match_id"]      
       )

In [ ]:
px.box(ionbot, 
       y=["rt-pred-error"],
       color="rank",
       hover_data=["ionbot_match_id"]
      )

In [ ]:
fig = px.scatter(ionbot, 
                 x="observed_retention_time", 
                 y="predicted_retention_time",
                 color="rank",
                 hover_data=["ionbot_match_id"]
                )
fig.update_traces(marker=dict(size=2))
fig.show()

In [ ]:
fig = px.scatter(ionbot, 
                 x="corrected_retention_time", 
                 y="predicted_retention_time",
                 color="rank",
                 hover_data=["ionbot_match_id"]
                )
fig.update_traces(marker=dict(size=2))
fig.show()

The `proteins` column contains detailed protein matching information:

In [ ]:
ionbot[["ionbot_match_id","proteins"]]

## Adding Uiversal Spectrum Identifiers

If the spectrum files were uploaded to a public ProteomeXchange repository, then PSM annotations can be obtained by adding Universal Spectral Identifiers (USI).

The USI is a proposed standard in the process of being ratified by the Proteomics Standards Initiative (PSI) that enables the identification of a specific spectrum or PSM contained in public ProteomeXchange repositories.

For more information, including the draft specification, please see http://psidev.info/usi/

The resuired url can be constructed from the columns in the results files:

In [ ]:
dataset = "PXD008601"

def get_universal_link(x):
    file = '.'.join(x["spectrum_file"].split('.')[:-1])
    s = x["matched_peptide"]
    if str(x["modifications"]) != "nan":
        tmp = x["modifications_delta"].split("|")
        seq = list(x["matched_peptide"])
        for i in range(0,len(tmp),2):
            pos = int(tmp[i])
            delta = tmp[i+1]
            if not delta.startswith('-'):
                delta = '%2B' + delta
            if pos == 0: #N-TERM
                seq.insert(pos,"[%s]"%delta)
            elif pos == len(seq)+1: #C-TERM
                seq.insert(pos-2,"[%s]"%delta)
            else:
                seq.insert(pos,"[%s]"%delta)
        s = ''.join(seq)
    link = "http://proteomecentral.proteomexchange.org/usi/?usi=mzspec:%s:%s:scan:%i:%s/%i"%(
        dataset,file,x["scan"],s,x["charge"])
    return f'<a target="_blank" href="%s">click</a>'%link

In [ ]:
ionbot["USI"] = ionbot.apply(get_universal_link,axis=1)

Now we added a column `USI` that contains links to the spectrum annotations:

In [ ]:
cols_to_use = ["ionbot_match_id","database_peptide","matched_peptide",
               "modifications","modifications_delta","unexpected_modification"]
ionbot[cols_to_use + ["USI"]]

Unfortunately this repository did not contain MGF files. So the USI will not work for visualizing the spectrum. The next link will take you to an example of a working USI visualization:
[link](http://proteomecentral.proteomexchange.org/usi/?usi=mzspec:PXD000561:Adult_Frontalcortex_bRP_Elite_85_f09:scan:17555:VLHPLEGAVVIIFK/2)                       

## JQuery Lorikeet PSM Annotations

Alternatively, PSM annotations can be computed from local MGF files:

In [ ]:
import annotations.lorikeet
import shutil

! mkdir mgf
! wget -q --no-parent https://genesis.ugent.be/uvpublicdata/copenhagen_20220913/20150929_QE5_UPLC10_RJC_SA_Plaque6_01.zip ./
! unzip -q 20150929_QE5_UPLC10_RJC_SA_Plaque6_01.zip -d ~/mgf/

You need to specify the folder that contains the spectrum MGF files and a folder to store the annotated spectra that are written as HTML files:

In [ ]:
mgf_folder = "mgf/"
annotations_folder = "my_annotations/"

Next, you can specify the PSMs to annotate as follows (for each PSM the corresponding MGF file and the scan number needs to specified):

In [ ]:
to_annotate = [
    ["20150929_QE5_UPLC10_RJC_SA_Plaque6_01.mgf",12057],
    ["20150929_QE5_UPLC10_RJC_SA_Plaque6_01.mgf",12058]
]

The following code will create the PSM annotations:

In [ ]:
for mgf_file, scan in to_annotate:
    html_filename = annotations.lorikeet.generate_html(annotations_folder,mgf_folder,mgf_file,scan,ionbot,l_os="linux")
    print("Annotations written to %s"%html_filename)

In [ ]:
shutil.make_archive("my_annotations", "zip", "my_annotations")

## Modifications

The 'unexpected_modification' column only shows the matched unexpected modification, not the modifications set as varialbe (expected):

In [ ]:
ionbot[["ionbot_match_id","modifications","unexpected_modification"]]

All matched modifications are in the 'modifications' column. We can parse this column as follows:

In [ ]:
modifications = {}

def get_modifications(x):
    if str(x) == "nan":
        return
    tmp = x.split('|')
    for i in range(0,len(tmp),2):
        if not tmp[i+1] in modifications:
            modifications[tmp[i+1]] = 0
        modifications[tmp[i+1]] += 1
        
ionbot["modifications"].apply(get_modifications)
{k: v for k, v in sorted(modifications.items(), key=lambda item: item[1], reverse=True)}

## The protein results

There are two protein inference result files:

- ionbot.first.proteins.csv
- ionbot.coeluting.proteins.csv

The first file contains the protein statistics infered from the first ranked matched only. The second file containst the protein statistics infered from all co-eluting matches.

We will continue with the proteins infered from all co-eluting matches:

In [ ]:
proteins = pd.read_csv(f"{single_result_folder}/ionbot.coeluting.proteins.csv")

In [ ]:
for col in proteins.columns:
    print(col)

These are the columns (described [here](https://ionbot.cloud/help)):

The `protein_group` column is a concatenation of the proteins it contains (search for '__'):

In [ ]:
cols_to_use = ["ionbot_match_id","protein_group","protein","position_in_protein","uniprot_id"]
proteins[cols_to_use]

Notice how protein groups that contain more than protein are also split over the rows. This allows for the 'position_in_protein', 'uniprot_id', 'protein_length' and 'protein_description' to make sense.

However, we want to look at protein groups only, so we remove these duplicated rows:

In [ ]:
cols_to_use = ["ionbot_match_id","is_shared_peptide","protein_group","protein_group_q-value","protein_group_PEP"]
proteins = proteins[cols_to_use]
proteins.drop_duplicates(["ionbot_match_id","protein_group"],inplace=True)
proteins

PSMs matched with two or more protein groups are indicated in the `is_shared_peptide` column:

In [ ]:
print(proteins["is_shared_peptide"].value_counts())

We wil continue with non-shared peptide matches only (you can of course skip this step):

In [ ]:
proteins = proteins[proteins["is_shared_peptide"]==False]

Now we can count the number of (non-shared) PSMs in each protein group and add this as a column called `#PSMs`:

In [ ]:
tmp = proteins["protein_group"].value_counts().reset_index(level=0)
tmp.columns = ["protein_group","#PSMs"]
proteins = proteins.merge(tmp,on="protein_group",how="left")
proteins.drop_duplicates(["protein_group"])[["protein_group","protein_group_q-value","#PSMs"]]

We can then count then number of protein groups with a specific number of PSMs:

In [ ]:
tmp = proteins.drop_duplicates("protein_group")["#PSMs"].value_counts().reset_index(level=0)
fig = px.pie(tmp, values='#PSMs', names='index', title='#PSMs in protein group')
fig.update_traces(textposition='inside')
fig.show()

To compute counts at the peptide level we need to merge the `proteins` data with the `ionbot` data (we do this using the `ionbot_match_id` column:

In [ ]:
proteins = proteins.merge(ionbot,on="ionbot_match_id",how="left")

In [ ]:
proteins.columns

Now we can count the number of unique peptides in each protein group and add this as a column called `#peptides`:

In [ ]:
tmp = proteins.drop_duplicates("matched_peptide")["protein_group"].value_counts().reset_index(level=0)
tmp.columns = ["protein_group","#peptides"]
proteins = proteins.merge(tmp,on="protein_group",how="left")

In [ ]:
proteins[cols_to_use + ["#peptides"]]

In [ ]:
tmp = proteins.drop_duplicates("protein_group")["#peptides"].value_counts().reset_index(level=0)
fig = px.pie(tmp, values='#peptides', names='index', title='#Peptides in protein group')
fig.update_traces(textposition='inside')
fig.show()

We can also compute protein group specific features:

In [ ]:
cols = ["psm_score","all-count","by-intensity-pattern-correlation"]
metrics = ["min","max"]


feature_cols = []
for col in cols:
    for metric in metrics:
        feature_cols.append(col+"_"+metric)
        proteins[col+"_"+metric] = proteins.groupby('protein_group')[col].transform(metric)
        
feature_cols

In [ ]:
proteins[["protein_group"] + feature_cols]

## Analyzing all results

In this section we will analyze all the files present in this project. First we will read all the .csv files. Note that we will not unpack the twbx file now, but we will immediately read in the unzipped version of the first ranked PSMs.

In [ ]:
from collections import Counter
from itertools import chain

ionbot_all = []
for f in processed_files:
    f_name = os.path.join(f,"Data/ionbot_result/ionbot.first.csv")
    print(f"Reading... {f_name}")
    ionbot_all.append(pd.read_csv(f_name)) 

We have read in the tables (i.e., matrices) seperately in the previous step. In this step we will merge all of these tables together. We use axis=0 so we link the columns and add rows from the different tables.

In [ ]:
ionbot_all_df = pd.concat(ionbot_all,axis=0)

Next we will format all modifications so they are position independant (removing modification position).

In [ ]:
modifications = {}

def get_modifications(x):
    if str(x) == "nan":
        return []
    tmp = x.split('|')
    ret_mods = []
    for i in range(0,len(tmp),2):
        ret_mods.append(tmp[i+1])
    return ret_mods
        
ionbot_all_df["stripped_modifications"] = ionbot_all_df["modifications"].apply(get_modifications)


We can count modifications now using "Counter", but we have to do some 'magic' to make sure we do not ignore modifications that co-occur on the same PSM.

In [ ]:
modification_count = Counter(chain(*list(ionbot_all_df["stripped_modifications"])))

Lets have a look at the top 25 most common modifications for all runs:

In [ ]:
modification_count.most_common()[0:25]

Lets single out the modern plaque samples, these contain the label "_RJC_SA_Plaque" in their file name.

In [ ]:
selection_modern = [True if "_RJC_SA_Plaque" in f else False for f in ionbot_all_df["spectrum_file"]]
selection_old = [True if "_RJC_SA_Plaque" not in f else False for f in ionbot_all_df["spectrum_file"]]

Again we count the modifications, but this time split it into old and modern samples.

In [ ]:
counter_old = Counter(chain(*list(ionbot_all_df[selection_old]["stripped_modifications"])))
counter_modern = Counter(chain(*list(ionbot_all_df[selection_modern]["stripped_modifications"])))

For normalization we use a very simple (naive...) approach of sum normalisation where each count value of a sample is divided by the sum of all modification counts. For this we will calculate the sum of counts.

In [ ]:
sum_old = sum(counter_old.values())
sum_new = sum(counter_modern.values())

Next we will plot the top 10 most occuring modifications and their counts in a pie plot. How does the top 100 (or even 500) compare to the counts of the top 10?

In [ ]:
old_pds = pd.Series(dict(counter_old.most_common()[0:10]))
modern_pds = pd.Series(dict(counter_modern.most_common()[0:10]))

fig = px.pie(old_pds, values=old_pds, names=old_pds.index, title='Modification count old samples')
fig.update_traces(textposition='inside')
fig.show()

fig = px.pie(modern_pds, values=modern_pds, names=modern_pds.index, title='Modification count modern samples')
fig.update_traces(textposition='inside')
fig.show()

There are certain modifications we expect in old sample due to degradation of the proteins. What kind of modifications do you expect? And if we look below can you visually see the groups?

In [ ]:
selected_mod = "[35]Oxidation[P]"

def count_spec_mods(mods):
    return Counter(chain(*list(mods)))[selected_mod]

def count_all_mods(mods):
    return Counter(chain(*list(mods))).values()

group_mods = ionbot_all_df.groupby("spectrum_file")["stripped_modifications"].apply(count_spec_mods)
group_mods_sum = ionbot_all_df.groupby("spectrum_file")["stripped_modifications"].apply(count_all_mods).apply(sum)

fig = px.bar((group_mods/group_mods_sum)*100, x=["_".join(v.split("_")[-4:-1]) for v in group_mods.index], y=(group_mods/group_mods_sum)*100)
fig.show()

In [ ]:
selected_mod = "[7]Deamidated[N]"

group_mods = ionbot_all_df.groupby("spectrum_file")["stripped_modifications"].apply(count_spec_mods)
group_mods_sum = ionbot_all_df.groupby("spectrum_file")["stripped_modifications"].apply(count_all_mods).apply(sum)

fig = px.bar((group_mods/group_mods_sum)*100, x=["_".join(v.split("_")[-4:-1]) for v in group_mods.index], y=(group_mods/group_mods_sum)*100)
fig.show()

If you want you can select different modifications by replacing the value set for "selected_mod". The top 100 below can provide some hints on what values to use:

In [ ]:
counter_old.most_common()[0:100]

## Modification profile clustering

In this section we will cluster the samples using the modification profiles in T-SNE. In the paper this was done mostly based on species, but even without knowing the species the modification landscape can provide many clues on the specific samples. First we will define a color map for the cluster (file/sample linked to color).

In [ ]:
from sklearn.manifold import TSNE

color_sample_map = {
    '20150929_QE5_UPLC10_RJC_SA_Plaque1_01.mgf' : "blue",
    '20150929_QE5_UPLC10_RJC_SA_Plaque2_01.mgf' : "blue",
    '20150929_QE5_UPLC10_RJC_SA_Plaque3_01.mgf' : "blue",
    '20150929_QE5_UPLC10_RJC_SA_Plaque4_01.mgf' : "blue",
    '20150929_QE5_UPLC10_RJC_SA_Plaque5_01.mgf' : "blue",
    '20150929_QE5_UPLC10_RJC_SA_Plaque6_01.mgf' : "blue",
    '20150929_QE5_UPLC10_RJC_SA_Plaque7_01.mgf' : "blue",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_1000_01.mgf' : "grey",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_1001_01.mgf' : "grey",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_1002_01.mgf' : "grey",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_1003_01.mgf' : "grey",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_987_01.mgf' : "red",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_988_01.mgf' : "red",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_989_01.mgf' : "red",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_990_01.mgf' : "green",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_991_01.mgf' : "green",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_992_01.mgf' : "green",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_993_01.mgf' : "red",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_994_01.mgf' : "red",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_995_01.mgf' : "red",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_996_01.mgf' : "red",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_997_01.mgf' : "red",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_998_01.mgf' : "red",
    '20160531_QE5_nLC14_RJC_COLLAB_GeoG_999_01.mgf' : "green",
    '20160608_QE5_nLC14_RJC_COLLAB_GeoG_1004_01.mgf' : "green",
    '20160608_QE5_nLC14_RJC_COLLAB_GeoG_1005_01.mgf' : "red",
    '20160608_QE5_nLC14_RJC_COLLAB_GeoG_1006_01.mgf' : "red",
    '20160608_QE5_nLC14_RJC_COLLAB_GeoG_1007_01.mgf' : "green",
    '20160608_QE5_nLC14_RJC_COLLAB_GeoG_1008_01.mgf' : "green",
    '20160608_QE5_nLC14_RJC_COLLAB_GeoG_1009_01.mgf' : "green",
    '20160608_QE5_nLC14_RJC_COLLAB_GeoG_1010_01.mgf' : "red",
    '20160608_QE5_nLC14_RJC_COLLAB_GeoG_1011_01.mgf' : "grey",
    '20160701_QE3_nLC5_MEM_MODCALC_C1_01.mgf' : "black",
    '20160701_QE3_nLC5_MEM_MODCALC_C2B_01.mgf' : "black",
    '20160701_QE3_nLC5_MEM_MODCALC_C3A_01.mgf' : "black",
    '20160701_QE3_nLC5_MEM_MODCALC_C4A_01.mgf' : "black",
    '20160701_QE3_nLC5_MEM_MODCALC_C5B_01.mgf' : "black",
    '20160701_QE3_nLC5_MEM_MODCALC_C6B_01.mgf' : "black",
    '20160701_QE3_nLC5_MEM_MODCALC_C7B_01.mgf' : "black"
}

This time we are grouping by spectrum file and applying the counter:

In [ ]:
def count_all_mods_counter(mods):
    return Counter(chain(*list(mods)))

ionbot_counter_per_file = ionbot_all_df.groupby("spectrum_file")["stripped_modifications"].apply(count_all_mods_counter)

We need a matrix for T-SNE, in the following cell we transform the grouped dataframe to a matrix dataframe: 

In [ ]:
mod_dict = {}
for k,v in ionbot_counter_per_file.to_dict().items():
    try:
        mod_dict[k[0]]
    except KeyError:
        mod_dict[k[0]] = {}
    mod_dict[k[0]][k[1]] = v
mod_df = pd.DataFrame(mod_dict)

Making sure there are not zeros in the matrix and fitting T-SNE.

In [ ]:
mod_df.fillna(0,inplace=True)

mod_df_embedded = TSNE(n_components=2, learning_rate='auto',
                       init='random', perplexity=30).fit_transform(mod_df.T)

As you can see the clustering is quite good. Can we conclude from this that the MODCALC (calcium deposites modern teeth) samples are closer to modern plaque samples than to the ancient ones? 

In [ ]:
plt.scatter(mod_df_embedded[:,0],mod_df_embedded[:,1], c = color_sample_map.values())